## Install the required libraries

In [1]:
!pip install sounddevice
!pip install pathlib


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pathlib from https://files.pythonhosted.org/packages/78/f9/690a8600b93c332de3ab4a344a4ac34f00c8f104917061f779db6a918ed6/pathlib-1.0.1-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import the required libraries

In [2]:
import numpy as np
import sounddevice as sd
import scipy.signal as sg
import matplotlib.pyplot as plt
from pathlib import Path

## Setting sweep parameters
- $F_s$ is the sampling frequency
- $f_1$ is the starting frequency
- $f_2$ is the ending frequency
- $T$ is the sampling duration
- $N$ is the number of samples
- $R = \ln{\frac{f_2}{f_1}}$

In [3]:
Fs = 48000
f1 = 100
f2 = 22000
T = 8
N = int(Fs * T)
R = np.log(f2 / f1)
n = np.arange(N)

## Sweep generation
- Generate the exponential sweep using $s[n]=\sin\bigg(\frac{2\pi f_1 T}{R}\bigg(\exp{\bigg(\frac{nR}{N}\bigg)}-1\bigg)\bigg)$

In [4]:
sweep = np.sin(2*np.pi*f1*T/R * (np.exp(n*R/N) - 1))
sweep *= sg.tukey(N, 0.04)    
sweep *= 10**(-3/20)               

## Probe building
- The sweep is repeated for channel synchronisation, by introducing two peaks that can be used for estimating the SRO

In [5]:
barker = np.array([1,1,1,1,1,0,0,1,1,0,1,0,1])*2-1 
barker = np.repeat(barker, 300) * 0.7        
probe  = np.hstack([barker, sweep, sweep])

## Play the probe

In [6]:
print("Playing 2× sweep (≈%5.1f s)…" % (probe.size/Fs))
sd.play(probe.astype(np.float32), Fs, blocking=True)
print("Done.")

Playing 2× sweep (≈ 16.1 s)…
Done.
